In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
import tensorflow as tf
import os
import image
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

2023-06-26 15:32:56.403106: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-26 15:32:56.503037: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-26 15:32:58.061172: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
file = 'optdigits.tra'
tr_data = pd.read_csv(file, header = None)
tr_data.head()
print(tr_data.iloc[:, -1].value_counts()) # target
print(tr_data.shape)

64
1    389
3    389
7    387
4    387
9    382
2    380
8    380
6    377
0    376
5    376
Name: count, dtype: int64
(3823, 65)


In [3]:
file1 = 'optdigits.tes'
te_data = pd.read_csv(file1, header = None)
te_data.head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,0,0,5,13,9,1,0,0,0,0,...,0,0,0,6,13,10,0,0,0,0
1,0,0,0,12,13,5,0,0,0,0,...,0,0,0,0,11,16,10,0,0,1
2,0,0,0,4,15,12,0,0,0,0,...,0,0,0,0,3,11,16,9,0,2
3,0,0,7,15,13,1,0,0,0,8,...,0,0,0,7,13,13,9,0,0,3
4,0,0,0,1,11,0,0,0,0,0,...,0,0,0,0,2,16,4,0,0,4


In [9]:
# RNN용

#X_train = np.expand_dims(tr_data.iloc[:, :-1], -1)
#X_test = np.expand_dims(te_data.iloc[:, :-1], -1)
#y_train = np.expand_dims(tr_data.iloc[:, -1], -1)
#y_test = np.expand_dims(te_data.iloc[:, -1], -1)


# DNN용
X_train = tr_data.iloc[:, :-1]
X_test = te_data.iloc[:, :-1]
y_train = tr_data.iloc[:, -1]
y_test = te_data.iloc[:, -1]

print(X_train.shape)
print(y_train.shape)

(3823, 64)
(3823,)


In [10]:
m = keras.Sequential([
    keras.layers.Dense(64, input_shape = (64,), activation = 'relu'),
    #keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(10)
])

m.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.001),
          loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics = ['accuracy'])

In [11]:
m.fit(X_train, y_train, epochs = 10)
m.evaluate(X_test, y_test)

Epoch 1/10


2023-06-26 15:36:18.484083: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fb97001fbc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-26 15:36:18.484118: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2023-06-26 15:36:18.512017: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-06-26 15:36:18.892422: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2023-06-26 15:36:19.190645: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


120/120 [==============================] - 3s 5ms/step - loss: 2.3287 - accuracy: 0.5572
Epoch 2/10
120/120 [==============================] - 0s 4ms/step - loss: 0.3192 - accuracy: 0.9032
Epoch 3/10
120/120 [==============================] - 0s 4ms/step - loss: 0.1968 - accuracy: 0.9427
Epoch 4/10
120/120 [==============================] - 0s 4ms/step - loss: 0.1400 - accuracy: 0.9579
Epoch 5/10
120/120 [==============================] - 0s 4ms/step - loss: 0.1105 - accuracy: 0.9678
Epoch 6/10
120/120 [==============================] - 0s 4ms/step - loss: 0.0853 - accuracy: 0.9772
Epoch 7/10
120/120 [==============================] - 0s 4ms/step - loss: 0.0692 - accuracy: 0.9814
Epoch 8/10
120/120 [==============================] - 0s 4ms/step - loss: 0.0618 - accuracy: 0.9822
Epoch 9/10
120/120 [==============================] - 0s 4ms/step - loss: 0.0505 - accuracy: 0.9880
Epoch 10/10
57/57 [==============================] - 0s 3ms/step - loss: 0.1664 - accuracy: 0.9510


[0.16643327474594116, 0.9510294795036316]

In [13]:
from sklearn.linear_model import LogisticRegression

m = LogisticRegression(solver='liblinear')
m.fit(X_train, y_train)
print(m.score(X_train, y_train))
print(m.score(X_test, y_test))

0.9869212660214491
0.9471341124095715


In [15]:
from sklearn.decomposition import PCA

p = PCA(n_components=8)
p = p.fit(X_train)
p_result = p.transform(X_train)
print(X_train.shape)
print(p_result.shape)

(3823, 64)
(3823, 8)


In [17]:
m = LogisticRegression(solver='liblinear')
m.fit(p_result, y_train)
print(m.score(p_result, y_train))

p_result_test = p.transform(X_test)
print(m.score(p_result_test, y_test))
#print(m.score(x_test, y_test))

0.9149882291394194
0.8759042849193099
